# Source: Wikipedia - All Americans

In [ ]:
#hide
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
import queries

# Set Notebook Settings

In [ ]:
#years = cc.get_defYears()
years = ['2021']
headers= cc.get_header()
dataset = 'AllAmerican'

# Get, Process the All American data from Wikipedia
#### Source: https://www.pro-football-reference.com/years/2017/draft.htm
> This page contains metadata of each draft pick, both in terms of draft position but also current pro stats.

In [ ]:
with open("..//scrapedData//aaSelections_2021.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.handle_allAmerican(years, headers)))

## Get Data from CSV
> Only necessary for 2009 because the Wiki page is a table not a list

In [ ]:
records = fx.get_csvAllAmerican('..//ScrapedData//aa_2009.csv')

In [ ]:
#process the records
pRecords = fx.process_csvAllAmerican(records, 2009)

In [ ]:
#save the processed records to the database
fx.toDB_csvAllAmerican(pRecords)

In [ ]:
# you have a duplicates issue with All American
# Can be seen here: select *  from SourcedPlayers where KeyDataSet = 6 group by ID, College Having count(*) > 1

# Clear DB
> Useful for a clean start.  This removes all of the records for this dataset from the following structures: SourcedPlayers, RecordLinks.  All of the Views auto-cleanse themselves.

In [ ]:
fx.clearDB(dataset)

# Save to DB
> This saves it to the SourcedPlayers Table

In [ ]:
fx.toDB_AllAmerican()

# Strict Matching
> This saves it to RecordLinking where ID == ID

In [ ]:
fx.literalLinking(dataset)

# Fuzzy Matching w/ Threshold

> This is automatically pushing fuzzy matches above a certain threshold into the DB without the need for review

In [ ]:
fuzzyDF = fx.doFuzzyMatching(dataset, 'Sports247', .78)

# Create the Annotation File

> This changes the dataframe into a MultiIndex data frame that the annotation function requires

In [ ]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pd.read_sql_query ('''
                               SELECT
                                   a.IDYR,
                                   a.PlayerName,
                                   a.College,
                                   a.Year
                               FROM SourcedPlayers as a
                                   inner join Positions as b
                                       ON a.Position = b.Position
                               WHERE a.KeyDataSet = 1
                              
                               ''', conn)

df_247 = pd.DataFrame(sql_query, columns = ['IDYR', 'PlayerName', 'College', 'Year'])
df_247.set_index('IDYR', append=False, inplace=True)

sql_query = pd.read_sql_query (queries.get_query_UnlinkedAllAmerican(), conn)
df_AllAmerican = pd.DataFrame(sql_query, columns = ['ID', 'PlayerName', 'Year', 'College'])
df_AllAmerican.set_index('ID', append=False, inplace=True)

fx.create_AnnotationFile(fuzzyDF, df_AllAmerican, df_247)

# Read in the Annotation File
> Take the resulting Annotation file after handling the processing and insert it into the right table

In [ ]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

## Insert Annotations to RecordLinks

In [ ]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 6, 1, 1, 1, 0]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, TargetKeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()

## Link to Rivals

In [ ]:
conn = sql.connect(cc.databaseName)
c = conn.cursor()
print("Connected to SQLite")

In [ ]:
fetchIds = c.execute('SELECT a.ID, b.ID from UnlinkedAllAmerican a inner join UnlinkedNCAA b on a.ID = b.ID')
records = c.fetchall()

In [ ]:
print(len(records))

In [ ]:
for record in records:
    #below you are hardcoding the KeyLinkType - this should probably be a lookup so it doesn't break in the future
    #if i'm working with All American data, as an example, then the MasterID is going to be the data source's unique ID and target will be 247
    sqlite_insert_query = """INSERT INTO RecordLinks
                        (MasterID, TargetID, KeyDataSet, TargetKeyDataSet, KeyLinkType, LinkConfidence) 
                        VALUES 
                        (?,?,?,?,2,1);"""
    data_tuple = [record[0],record[1],6,5]
    count = c.execute(sqlite_insert_query, data_tuple)
    conn.commit()